In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/semeval_2022/maMi

/content/drive/My Drive/semeval_2022/maMi


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 8.0 MB/s 
     |████████████████████████████████| 895 kB 50.8 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 68.2 MB/s 
     |████████████████████████████████| 3.3 MB 58.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 8.0 MB/s 


In [ ]:
import os
import re
import numpy as np
import string
import json
#import emoji
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
#from transformers import BigBirdTokenizer, BigBirdModel, AdamW
from transformers import AdamW, XLNetTokenizer, XLNetModel, BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel,RobertaModel,RobertaConfig
pd.set_option("display.max_columns", None)
import random
from ast import literal_eval

In [ ]:
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

In [ ]:
random.seed(55)
np.random.seed(55)
torch.manual_seed(55)
torch.cuda.manual_seed_all(55)

In [ ]:
test_df=pd.read_csv('./test/Test.csv',sep='\t')
test_df['Text Transcription']=test_df['Text Transcription'].apply(lambda z: remove_punctuations(z))
test_df['Text Transcription']=test_df['Text Transcription'].apply(lambda z: remove_html(z))
test_df['Text Transcription']=test_df['Text Transcription'].apply(lambda z: remove_url(z))
test_df.rename(columns={'Text Transcription': 'text'}, inplace=True)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 32

In [ ]:
MAX_LEN = 50
tokenizer1 = AutoTokenizer.from_pretrained('roberta-base')
tokenizer2 = XLNetTokenizer.from_pretrained('xlnet-base-cased')
target_cols=['shaming','stereotype','objectification','violence']

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [ ]:
class BERTDatasetest(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.text
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

In [ ]:
test_dataset=BERTDatasetest(test_df,tokenizer2,MAX_LEN)
test_loader = DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, 
                          num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
config = RobertaConfig.from_json_file('./checkpoint-25000/config.json')

In [ ]:
class BERTClass2(torch.nn.Module):
    def __init__(self):
        super(BERTClass2, self).__init__()
        #self.roberta = BigBirdModel.from_pretrained('google/bigbird-roberta-base')
        self.roberta = XLNetModel.from_pretrained('xlnet-base-cased')
        self.l2 = torch.nn.Dropout(0.2)
        self.fc = torch.nn.Linear(768,4)
    
    def forward(self, ids, mask, token_type_ids):
       # _,features = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        last_hidden_state = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        mean_hidden_state=self.pool_hidden_state(last_hidden_state)
        mean_hidden_state = self.l2(mean_hidden_state)
        output = self.fc(mean_hidden_state)
        return output
    def pool_hidden_state(self, last_hidden_state):
        """
        Pool the output vectors into a single mean vector 
        """
        last_hidden_state = last_hidden_state[0]
        mean_last_hidden_state = torch.mean(last_hidden_state, 1)
        return mean_last_hidden_state

In [ ]:
class BERTClass1(torch.nn.Module):
    def __init__(self):
        super(BERTClass1, self).__init__()
        #self.roberta = AutoModel.from_pretrained('roberta-base')
        self.roberta = RobertaModel.from_pretrained('./checkpoint-25000/',local_files_only=True,config=config)
#         self.l2 = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(768,4)
    
    def forward(self, ids, mask, token_type_ids):
        _, features = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
#         output_2 = self.l2(output_1)
        output = self.fc(features)
        return output

In [ ]:
model1 = BERTClass1()
model1.to(device)



Some weights of the model checkpoint at ./checkpoint-25000/ were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ./checkpoint-25000/ and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it

BERTClass1(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,)

In [ ]:
model2 = BERTClass2()
model2.to(device)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass2(
  (roberta): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [ ]:
checkpoint1=torch.load('./saved_models/roberta_pre_50_10_whole.pt')
model1.load_state_dict(checkpoint1['state_dict'])



<All keys matched successfully>

In [ ]:
checkpoint2=torch.load('./saved_models/xlnet_mami_50_10_16_full.pt')
model2.load_state_dict(checkpoint2['state_dict'])

<All keys matched successfully>

In [ ]:
def test():
    model2.eval()
    fin_outputs=[]
    with torch.no_grad():
      for _, data in enumerate(test_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        outputs = model2(ids, mask, token_type_ids)
        fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs

In [ ]:
def test_soft():
    model2.eval()
    fin_outputs=[]
    with torch.no_grad():
      for _, data in enumerate(test_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        outputs = model2(ids, mask, token_type_ids)
        fin_outputs.extend(outputs.cpu().detach().numpy().tolist())
    return fin_outputs

In [ ]:
outputs1=test()

In [ ]:
outputs2=test()

In [ ]:
outputs1=test_soft()

In [ ]:
outputs2=test_soft()

In [ ]:
#for voting ensemble
outputs1 = np.clip(np.sign(np.array(outputs1)), a_min=0,a_max=None)
outputs2 = np.clip(np.sign(np.array(outputs2)), a_min=0,a_max=None)

In [ ]:
from scipy.stats import stats

In [ ]:
def ensemble_posteriors(posteriors):
    avg_posteriors = np.mean(np.stack(posteriors, axis=0), axis=0)
    return avg_posteriors

In [ ]:
def ensemble_voting(predictions):
    stacked = np.stack(predictions, axis=0)
    modals = stats.mode(stacked, axis=0)[0].squeeze().astype(int)
    return modals

In [ ]:
##for voting ensemble
outputs=ensemble_voting([outputs1,outputs2])

In [ ]:
avg_posteriors=ensemble_posteriors([outputs1,outputs2])

In [ ]:
avg_posteriors

array([[0.00967844, 0.51380759, 0.02465093, 0.02988673],
       [0.77481166, 0.99312904, 0.01063315, 0.02155016],
       [0.01426466, 0.48041683, 0.09253167, 0.00177206],
       ...,
       [0.56478526, 0.05977175, 0.99720466, 0.02574177],
       [0.00117334, 0.96315616, 0.00285619, 0.07315265],
       [0.00152785, 0.02984747, 0.00182755, 0.00454194]])

In [ ]:
outputs = np.array(avg_posteriors) >= 0.5
outputs=np.round(outputs)

In [ ]:
outputs = np.clip(np.sign(np.array(avg_posteriors)), a_min=0,a_max=None)

In [ ]:
outputs

array([[0., 1., 0., 0.],
       [1., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [1., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.]], dtype=float16)

In [ ]:
outputs=np.round(outputs)

In [ ]:
predictions_db = pd.DataFrame(outputs,  columns=['shaming','stereotype','objectification','violence'])
predictions_db = predictions_db.apply(lambda x:  list(map(int,x)))
predictions_db['file_name']=test_df['file_name']


In [ ]:
for i in predictions_db.index:
  if predictions_db.loc[i,'shaming']==1 or predictions_db.loc[i,'stereotype']==1 or predictions_db.loc[i,'objectification']==1 or predictions_db.loc[i,'violence']==1:
    predictions_db.loc[i,'misogynous']=1
  else:
    predictions_db.loc[i,'misogynous']=0
predictions_db = predictions_db[['file_name','misogynous', 'shaming','stereotype','objectification','violence']]
predictions_db['misogynous']=predictions_db['misogynous'].apply(int)

In [ ]:
predictions_db.head()

,file_name,misogynous,shaming,stereotype,objectification,violence
0,15236.jpg,1,0,1,0,0
1,15805.jpg,1,1,1,0,0
2,16254.jpg,0,0,0,0,0
3,16191.jpg,1,1,1,1,0
4,15952.jpg,1,0,1,0,0


In [ ]:
predictions_db.to_csv('./final_res/answer21.txt', index=False, sep='\t', header=False)

In [ ]:
outputs= np.array(avg_posteriors) >= 0.35

In [ ]:
f1_score_micro = metrics.f1_score(targets, final_op, average='micro')
f1_score_macro = metrics.f1_score(targets, final_op, average='macro')
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.7203791469194313
F1 Score (Macro) = 0.5627604218996682


In [ ]:
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.7139689578713968
F1 Score (Macro) = 0.5728059046791082
